In [262]:
import pandas as pd
import json
import datetime

### Import Data and Mapping File

In [441]:
with open("../data/mapping/attration.json", "r") as file:
    mapping = json.load(file)

df = pd.read_excel("../data/attractions/Location-Import.xlsx", sheet_name="Landmark")
df.head(5)

,LPNumber,LandmarkType,DesignationDate,Id
0,LP-01962,Individual Landmark,1997-06-17,6451daad3879cf70e53ff07d
1,LP-01963,Individual Landmark,1997-06-17,560dd46ef89701aedbaebbf2
2,LP-01964,Individual Landmark,1997-09-16,64876a723ee0be65782261a2
3,LP-01965,Individual Landmark,1998-04-14,644592b4e7abb0665474a217
4,LP-01966,Individual Landmark,1999-08-03,648677a63ee0be657822618d


#### Clean Dataframe

In [442]:
def clean_dataframe(df):
    """
    Cleans the DataFrame based on a set of defined rules.

    Args:
    - df (pd.DataFrame): The DataFrame to clean.

    Returns:
    - pd.DataFrame: The cleaned DataFrame.
    """
    # Rule 1: Trim leading and trailing whitespace from all string values
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
        
    return df

# Use the function
df = clean_dataframe(df)

### Validate Data 

In [443]:
import re

def validate_dataframe(df, mapping):
    """
    Validates the DataFrame based on the given mapping.

    Args:
    - df (pd.DataFrame): The DataFrame containing the data.
    - mapping (dict): The mapping containing validation rules.

    Returns:
    - dict: A dictionary with keys as column names and values as lists of invalid entries for that column.
    """
    validation_errors = {}

    # TODO: Add more validation rules as needed based on the mapping
    for field, rules in mapping.items():
        # Check if the field is present in the DataFrame
        if rules.get('column') in df.columns:
            column_name = rules['column']
            # Validate based on string pattern
            if rules.get('type') == 'string' and rules.get('validation') and 'pattern' in rules['validation']:
                pattern = re.compile(rules['validation']['pattern'])
                invalid_values = [value for value in df[column_name] if not pattern.match(str(value))]
                if invalid_values:
                    validation_errors[column_name] = invalid_values

    return validation_errors


# Validate the dataframe
errors = validate_dataframe(df, mapping)

# Display validation errors
for column, invalid_values in errors.items():
    print(f"Invalid values for {column}: {invalid_values}")

# Display first 5 rows of the dataframe
df.head(5)

,LPNumber,LandmarkType,DesignationDate,Id
0,LP-01962,Individual Landmark,1997-06-17,6451daad3879cf70e53ff07d
1,LP-01963,Individual Landmark,1997-06-17,560dd46ef89701aedbaebbf2
2,LP-01964,Individual Landmark,1997-09-16,64876a723ee0be65782261a2
3,LP-01965,Individual Landmark,1998-04-14,644592b4e7abb0665474a217
4,LP-01966,Individual Landmark,1999-08-03,648677a63ee0be657822618d


In [444]:
import datetime

def process_dataframe_based_on_mapping(df, mapping):
    output = []

    for _, row in df.iterrows():
        record = {}
        for key, value in mapping.items():
            if isinstance(value, dict) and "column" in value:
                column_name = value["column"]
                if column_name in df.columns:
                    if "documentField" in value:
                        doc_field = value["documentField"]
                        if isinstance(row[column_name], pd.Timestamp):
                            # Special handling for date columns to represent in MongoDB Extended JSON format
                            date_str = row[column_name].strftime('%Y-%m-%dT%H:%M:%S')
                            record[doc_field] = {"$date": date_str}
                        else:
                            record[doc_field] = row[column_name]
        output.append(record)

    return output

result = process_dataframe_based_on_mapping(df, mapping)

current_datetime = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
output_path = f"../data/attractions/Location-Import_{current_datetime}.json"

with open(output_path, "w") as file:
    json.dump(result, file, indent=4)

print(output_path) 



../data/attractions/Location-Import_2023_09_30_23_11_46.json


In [ ]:
print(json.dumps(result, indent=4)) 